In [2]:
import pandas as pd

# ==== RUTAS ====
PATH_MOVIEGENRE = "MovieGenre.csv"
PATH_LINKS      = "links.csv"
OUT_PATH        = "merged_movies_genres.csv"

# ==== Lector con fallback de encoding ====
def read_csv_fallback(path, **kwargs):
    encodings = ["utf-8", "latin-1", "cp1252"]
    last_err = None
    for enc in encodings:
        try:
            return pd.read_csv(path, encoding=enc, **kwargs)
        except UnicodeDecodeError as e:
            last_err = e
    # Si todas fallan, re-lanza el último error
    raise last_err

# ==== Cargar (sin cambiar imdbId) ====
mg = read_csv_fallback(PATH_MOVIEGENRE)
ln = read_csv_fallback(PATH_LINKS)

print("Columnas MovieGenre:", list(mg.columns))
print("Columnas links:", list(ln.columns))

# ==== Detectar columnas imdb (sin renombrar ni transformar valores) ====
def find_imdb_col(df):
    # Preferir 'imdbId' exacto; si no existe, buscar algo que contenga 'imdb'
    cols_lower = {c.lower(): c for c in df.columns}
    if "imdbid" in cols_lower:
        return cols_lower["imdbid"]
    # fallback: primera columna que contenga 'imdb'
    for c in df.columns:
        if "imdb" in c.lower():
            return c
    raise ValueError("No se encontró una columna con 'imdb' en: " + str(list(df.columns)))

mg_imdb_col = find_imdb_col(mg)
ln_imdb_col = find_imdb_col(ln)

print("Usando columnas:", mg_imdb_col, "(MovieGenre)  y", ln_imdb_col, "(links)")

# ==== Merge exacto por imdbId como texto (sin alterar el valor original) ====
# Creamos llaves temporales solo para el join (no se guardan)
mg_key = mg[mg_imdb_col].astype(str)
ln_key = ln[ln_imdb_col].astype(str)

merged = mg.assign(_key=mg_key).merge(
    ln.assign(_key=ln_key),
    on="_key",
    how="inner",
    suffixes=("_mg", "_ln")
).drop(columns=["_key"])

print(f"Filas MovieGenre: {len(mg)} | links: {len(ln)} | tras merge: {len(merged)}")

# (Opcional) Reordenar: poner primero las columnas imdb detectadas
first_cols = []
for c in [mg_imdb_col, ln_imdb_col]:
    if c in merged.columns and c not in first_cols:
        first_cols.append(c)
other_cols = [c for c in merged.columns if c not in first_cols]
merged = merged[first_cols + other_cols]

# ==== Guardar ====
merged.to_csv(OUT_PATH, index=False)
print("✅ Merge guardado en:", OUT_PATH)

# Vistazo
display(merged.head())


Columnas MovieGenre: ['imdbId', 'Imdb Link', 'Title', 'IMDB Score', 'Genre', 'Poster']
Columnas links: ['movieId', 'imdbId', 'tmdbId']
Usando columnas: imdbId (MovieGenre)  y imdbId (links)
Filas MovieGenre: 40108 | links: 62423 | tras merge: 37416
✅ Merge guardado en: merged_movies_genres.csv


,imdbId_mg,Imdb Link,Title,IMDB Score,Genre,Poster,movieId,imdbId_ln,tmdbId
0,114709,http://www.imdb.com/title/tt114709,Toy Story (1995),8.3,Animation|Adventure|Comedy,https://images-na.ssl-images-amazon.com/images...,1,114709,862.0
1,113497,http://www.imdb.com/title/tt113497,Jumanji (1995),6.9,Action|Adventure|Family,https://images-na.ssl-images-amazon.com/images...,2,113497,8844.0
2,113228,http://www.imdb.com/title/tt113228,Grumpier Old Men (1995),6.6,Comedy|Romance,https://images-na.ssl-images-amazon.com/images...,3,113228,15602.0
3,114885,http://www.imdb.com/title/tt114885,Waiting to Exhale (1995),5.7,Comedy|Drama|Romance,https://images-na.ssl-images-amazon.com/images...,4,114885,31357.0
4,113041,http://www.imdb.com/title/tt113041,Father of the Bride Part II (1995),5.9,Comedy|Family|Romance,https://images-na.ssl-images-amazon.com/images...,5,113041,11862.0


In [3]:
import pandas as pd

IN_PATH  = "merged_movies_genres.csv"   # salida previa
OUT_PATH = "merged_movies_genres.csv"   # sobrescribir (o cambia el nombre si prefieres)

df = pd.read_csv(IN_PATH)

# === localizar la columna movieId (exacta o con sufijos del merge) ===
candidates = [
    "movieId",           # nombre típico en links.csv
    "movieId_ln",        # si vino desde links con sufijo
    "movieId_mg",        # si vino desde MovieGenre con sufijo
]

movie_col = next((c for c in candidates if c in df.columns), None)
if movie_col is None:
    # búsqueda flexible por si cambió el casing
    for c in df.columns:
        if c.lower() == "movieid":
            movie_col = c
            break

if movie_col is None:
    raise ValueError(f"No encontré columna 'movieId' (ni variantes). Columnas: {list(df.columns)}")

# === reordenar: movieId primero, el resto después ===
new_cols = [movie_col] + [c for c in df.columns if c != movie_col]
df = df[new_cols]

df.to_csv(OUT_PATH, index=False)
print(f"✅ Reordenado y guardado en: {OUT_PATH}")
print("Primera fila (verificación rápida):")
print(df.head(1))


✅ Reordenado y guardado en: merged_movies_genres.csv
Primera fila (verificación rápida):
   movieId  imdbId_mg                           Imdb Link             Title  \
0        1     114709  http://www.imdb.com/title/tt114709  Toy Story (1995)   

   IMDB Score                       Genre  \
0         8.3  Animation|Adventure|Comedy   

                                              Poster  imdbId_ln  tmdbId  
0  https://images-na.ssl-images-amazon.com/images...     114709   862.0  
